In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import ast
import re
import math

!! pip install geopy
from geopy.geocoders import Nominatim
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
df = pd.read_csv(r"C:\Users\dell\Desktop\Data Analyst Project\zomato2\zomato.csv", encoding='utf-8')

Lets drop duplicates based on the value from two columns address and name

In [6]:
df = df.drop_duplicates(subset=['address', 'name']).reset_index().drop('index', axis=1)
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


First let us make food corpus for our data

Since the column reviews_list, menu_item are in the form of list of tuples but as we read it from file we need to explicitly convert it to list

In [7]:
df['reviews_list'] =  df['reviews_list'].apply(ast.literal_eval)
df['menu_item'] =  df['menu_item'].apply(ast.literal_eval)

In [9]:
li = []
def func(x):
    global li
    li = li + x
a = df[df['menu_item']!="[]"]['menu_item'].apply(func)

In [10]:
# convert all the menu to lower case letter and storing in the variable food_corpus
food_corpus = list(map(str.lower, set(li)))

In [ ]:
## food_corpus

On analysing the corpus it was found that some food contains quantity written beside it inside the brackets.
Let us remove those brackets.

In [11]:
menu_corpus = []
for x in food_corpus:
    menu_corpus.append(re.sub("\[.*\]|\(.*\)", "", x).strip())

In [12]:
len(menu_corpus)

71424

Very well, so we have 71,424 food items in our corpus. But there is no data without noise, let us remove noise from it.

Lets analyse the data, for the words that frequently occur and are not a food item.

In [15]:
ind = df[df['reviews_list']!="[]"].index
for i in ind:
    liked = set()
    for review in df.loc[i,'reviews_list']:
        if(float(review[0].replace('Rated',''))>=3):
            for menu in menu_corpus:
                if(menu in review[1]):
                    liked.add(menu)
    print(liked)
                

{'butter', 'roti', 'paratha.', 'fried rice', 'bread', 'butter roti', 'chilli paratha', 'masala papad', 'kulcha', 'palak', 'rice', 'paratha', 'soya chaap', 'papad', 'fried', 'baby corn', 'poi'}
{'undo', 'roti', 'sweet', 'chicken', 'paneer', 'egg', 'spicy', 'sweet and spicy', 'veggie', 'burnt garlic fried rice', 'garlic fried rice', 'veg', 'rice', 'chicken biryan', 'fried', 'mix veg', 'uttar dakshin', 'paneer kurchan', 'fried rice', 'lunch', 'chicken biryani'}
{'coconut milk', 'lasagne', 'nutella', 'sweet', 'egg', 'chocolate', 'hot chocolate', 'choco', 'spicy', 'disco', 'milk', 'sicilia pizza', 'hot choco', 'coffee', 'pizza', 'veggie', 'veg', 'churros', 'rice', 'cake', 'nachos', 'special', 'smile', 'pink pasta', 'chocolate sauce', 'veggies', 'shakes', 'pasta', 'water', 'poi'}
{'sambar', 'full meal', 'holige', 'full meals', 'sweet', 'pulav', 'rice bath', 'apple', 'pulka', 'limited meal', 'rasam', 'veg rice', 'payasam', 'veg', 'meal', 'rice with sambar', 'rice', 'dosa', 'pickle', 'ghee', '

{'panneer butter masala', 'sev', 'grill', 'latte', 'chapati', 'butter chicken curry', 'kabab', 'chicken gravy', 'chicken kadai', 'tandoori chicken', 'grilled chicken', 'prawns', 'paneer butter masala', 'butter masala', 'dal', 'lime soda', 'shawarma', 'rice', 'alfaham', 'shakes', 'butter', 'roti', 'chai', 'grape', 'coin parota', 'paneer butter', 'fish', 'chicken', 'lollipop', 'anjal', 'parota', 'spicy', 'roll', 'pepper chicken', 'veg lovers', 'chili chicken', 'paneer butter masala.', 'ghee', 'anjal tawa', 'lunch', 'grill chicken', 'kadai', 'fresh lime', 'chicken wings', 'special chicken', 'noodles', 'naan', 'fresh lime soda', 'chocolate', 'prawns tawa fry', 'choco', 'al faham', 'tawa fry', 'chilly chicken', 'veg', 'grape juice', 'chicken biryan', 'special', 'butter chicken', 'ghee rice', 'thali', 'chicken curry', 'kadai chicken', 'soda', 'prawns tawa', 'paneer', 'tandoori roti', 'lime', 'alfaham chicken', 'chicken skewer', 'pulpy grape juice', 'tandoori', 'mutton biryani', 'meal', 'dosa

{'butter', 'onion raitha', 'chicken', 'paneer', 'roasted', 'egg chilli', 'egg', 'spicy', 'poratta', 'mushroom fry', 'mint cooler', 'raitha', 'rice', 'onion', 'rabadi', 'fried', 'special', 'mushroom', 'tea', 'spice', 'poi'}
{'roll', 'chicken rolls', 'chicken', 'chicken roll'}
{'butter', 'veg red sauce pasta', 'roti', 'butter naan', 'khichdi', 'seu', 'chicken tikka', 'red sauce pasta', 'baby corn chilly', 'chicken noodles.', 'noodles', 'chicken', 'butter naan.', 'naan', 'paneer tikka', 'paneer', 'cold coffe', 'roomali roti', 'spicy', 'crispy babycorn', 'garlic butter naan', 'baby corn', 'cold coffee', 'dal', 'coffee', 'schezwan noodles', 'veg', 'bread', 'meal', 'rice', 'chicken noodles', 'garlic bread', 'dal khichdi', 'jamun', 'fried', 'special', 'veg pasta', 'non-veg', 'schezwan noodle', 'chicken tikka masala', 'curd rice', 'curd', 'white sauce pasta', 'fried rice', 'chicken noodle', 'kit kat', 'naan.', 'baby corn chil', 'kit kat shake', 'churi', 'pasta', 'water', 'poi', 'packing'}
{'ro

{'popcorn', 'chicken', 'chicken pop', 'chicken popcorn'}
set()
{'vanilla', 'kulfi', 'sweet', 'blueberry', 'chocolate', 'choco', 'roll', 'milk', 'wafer', 'coffee', 'rice', 'kesar kulfi', 'special', 'blueberry ice cream', 'lychee', 'chocolate fudge', 'ice cream', 'overload', 'banana', 'vanilla ice cream', 'fudge', 'chocolate sauce'}
{'butter', 'butter naan', 'biriyani rice.', 'noodles', 'chicken', 'naan', 'panner butter', 'prawns', 'prawns fried rice', 'chicken fried rice', 'butter masala', 'panner butter masala', 'rice', 'chicken noodles', 'fried', 'biriyani rice', 'fried rice', 'chicken noodle', 'prawns fried rice.'}
{'chai'}
set()
{'chicken rolls', 'chicken tikka', 'sweet', 'chicken', 'crispy chicken', 'chicken kabab.', 'kebab', 'kabab', 'roll', 'spicy', 'chicken gravy', 'tandoori chicken', 'shadi ka biryani', 'packing', 'tandoori kebab', 'tandoori', 'crispy chicken biryani', 'veg lovers', 'malai kofta', 'mutton biryani', 'lemon chicken', 'veg', 'meal', 'chicken roll', 'rice', 'onion'

{'chai', 'samosa', 'kulfi', 'sweet', 'puri', 'dahi puri', 'lime', 'milk', 'moti chur laddu', 'kachori', 'dahi', 'cap', 'special', 'laddu', 'badam milk', 'curd', 'badam', 'legendary', 'rasmalai', 'idli'}
{'cheese', 'mixed', 'mayo', 'spicy', 'baby corn', 'mixed sauce pasta', 'poi', 'mayonnaise', 'bread', 'rice', 'garlic bread', 'fried', 'smile', 'mushroom', 'cheese garlic bread', 'fried rice', 'mushroom pepper', 'fries', 'pasta', 'triple schezwan fried rice', 'schezwan fried rice'}
{'butter', 'akki rotti', 'butter fruit juice', 'rice bath', 'fruit juice', 'filter coffee', 'set dosa', 'lime', 'rava dosa', 'milk', 'pulao', 'sandwich', 'coffee', 'veg', 'butterscotch', 'butterscotch milkshake', 'rice', 'dosa', 'onion', 'butter fruit', 'capsicum', 'milkshake', 'cap', 'special', 'masala dosa', 'onion dosa', 'pongal', 'rawa dosa', 'poi'}
{'pizza', 'veg', 'burger', 'cheese', 'onion', 'capsicum', 'paneer', 'cap', 'sandwich'}
{'kulfi', 'sweet', 'lassi', 'hot jamun', 'latte', 'sarson ka saag', 'sal

...

{'butter', 'roti', 'butter naan', 'sev', 'chicken tikka', 'green peas', 'grill', 'grilled fish', 'fish', 'chicken', 'naan', 'pepper chicken masala', 'kebab', 'parota', 'chicken masala', 'mayo', 'spicy', 'kabab', 'tikka biryani', 'chicken gravy', 'rotis', 'grilled chicken', 'pepper chicken', 'chicken tikka biryani', 'veg lovers', 'chicken hyderabadi', 'veg', 'bread', 'rice', 'rabadi', 'toum', 'ghee', 'chicken hyderabad', 'hyderabadi biryani', 'ghee rice', 'non-veg', 'chicken hyderabadi biryani', 'lunch', 'grill chicken', 'hummus', 'churi', 'poi'}
{'butter', 'special chicken biryani', 'special chicken', 'sev', 'paneer butter', 'paneer tikka biryani', 'fish', 'chicken', 'chicken kebab', 'boiled egg', 'naan', 'paneer tikka', 'bamboo chicken', 'chicken starters', 'egg', 'brinjal biryani', 'kebab', 'paneer', 'tangdi kabab', 'veg & veg', 'ida', 'chicken crispy', 'veg thali', 'spicy', 'kabab', 'hulk', 'tikka biryani', 'lime', 'kpl', 'boiled egg.', 'garlic chicken', 'paneer butter masala', 'but

{'butter', 'butter naan', 'tea', 'chai', 'lunch', 'spice', 'veg', 'paneer masala', 'palak', 'naan.', 'butter paneer', 'naan', 'rice', 'paneer', 'special', 'butter paneer masala', 'butter naan.', 'palak paneer'}
set()
{'kabab', 'biryani rice', 'meal', 'rice', 'chicken', 'chilly chicken'}
{'hyderabadi mutton biryani', 'non veg combo', 'seu', 'sev', 'chicken tikka', 'kesari phirni', 'sweet', 'chicken', 'claypot hyderabadi chicken biryani', 'paneer tikka', 'paneer', 'hyderbadi biryani', 'ida', 'kesari', 'mixed', 'tikka biryani', 'packing', 'hyderabadi mutton', 'chicken tikka biryani', 'veg combo', 'boneless chicken tikka', 'veg', 'mutton biryani', 'meal', 'rice', 'chicken biryan', 'rabadi', 'special', 'jamun', 'mix veg', 'non-veg', 'gulab jamun', 'veg biryani', 'lunch', 'spice', 'hyderabadi chicken biryani', 'chicken biryani', 'hyderabadi chicken', 'paneer biryani', 'boneless chicken', 'mutton biriyani', 'phirni', 'poi', 'raita'}
{'butter', 'roti', 'butter naan', 'chicken biriyani', 'chick

{'butter', 'lasagne', 'chicken mac n cheese', 'bombay', 'chicken tikka', 'tandoori kukkad', 'cheese', 'paneer steak', 'chicken', 'cheesy', 'desi videsi', 'paneer', 'roasted', 'egg', 'chocolate', 'choco', 'mac n cheese', 'chocolate lover', 'bombay masala quesadilla', 'tandoori', 'veggie', 'chicken lasagna', 'lasagna', 'butter chicken mac n cheese', 'veg', 'minced chicken', 'meal', 'rice', 'butter chicken', 'tea', 'lunch', 'veggies', 'pasta', 'packing'}
{'butter', 'butter naan', 'chicken biriyani', 'chai', 'sweet', 'chicken', 'naan', 'roasted', 'spicy', 'tandoori chicken', 'mushroom masala', 'chilli chicken', 'tandoori', 'shawarma', 'veg lovers', 'veg', 'rice', 'special', 'mushroom', 'chicken curry', 'lunch', 'spice', 'mast', 'spicy shawarma', 'poi'}
{'mazza'}
{'roti', 'fanta', 'full meal', 'khichdi', 'paneer makhani', '2 bread', 'chicken tikka', 'pakoda', 'cheese', 'aloo paratha meal', 'sausages', 'sweet', 'chicken', 'aloo paratha', 'capsicum curry', 'sabji', 'cheesy', 'paneer', 'paneer

{'choco', 'cupcake', 'rocher', 'cake', 'chocolate'}
{'veggie', 'veg', 'noodles', 'veggies', 'egg'}
{'manchow', 'chicken manchow soup', 'kulcha', 'chicken', 'chilli prawns', 'kebab', 'chicken briyani', 'mixed', 'spicy', 'boneless chicken biryani', 'red hot chicken', 'prawns', 'baby corn', 'pepper chicken', 'chilli prawn', 'veg', 'rice', 'chicken biryan', 'rabadi', 'manchow soup', 'special', 'veg crispy', 'thali', 'lunch', 'family biryani', 'chicken manchow', 'chicken biryani', 'boneless chicken', 'poi'}
{'cool breeze', 'roti', 'baklava', 'chai', 'cheese spring roll', 'chicken tikka', 'cheese', 'fish', 'chicken', 'lassi', 'cock', 'sweet', 'sausage', 'classic chicken tikka', 'kebab', 'cutlet', 'popcorn', 'chicken wings', 'classic', 'roll', 'chicken pizza', 'fish tikka', 'peni', 'aqua', 'spinach and corn quiche', 'cherry', 'chilli chicken', 'pizza', 'chicken sausage', 'chicken cutlet', 'rice', 'roti.', 'cake', 'chilli cheese', 'ajwaini fish tikka', 'special', 'chicken bao', 'spring rolls',

AttributeError: 'NoneType' object has no attribute 'replace'

After analysing the above output these were some of the frequent non menu words found.

In [16]:
remove_words = ['poi', 'spite', 'churi', 'peni', 'packing', 'meal', 'malt', 'smilies', 'disco', 'tac', 'cap', 'spicy', 'last night', 'cop', 
'brain', 'onion', 'spice', 'holige', 'adas', 'sulaimani', 'special', 'combo 4', 'toppings', 'smile', 'water', 'ghee', '',
'mixed', 'slice', 'lunch', 'masala c', 'churi', 'breakfast combo', 'undo', 'full meal', 'fanta', 'cool blue', 'love', 'veg lovers',
'irish', 'fried', 'mast', 'rose', 'overload']
# Lets remove them from our corpus first
new_menu_corpus = [word for word in menu_corpus if word not in remove_words]
len(new_menu_corpus)

71363

In [17]:
# perfect now those noise are removed, let us extract menus from review and store it in the column liked_food_from_review

df['liked_food_from_review'] = df['liked_food_from_review'].astype(object)
df['liked_food_from_review'] = "[]"

KeyError: 'liked_food_from_review'